<a href="https://colab.research.google.com/github/JohnnyHanun/Hw4/blob/main/Hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install facenet-pytorch mmcv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 13.3 MB/s 
     |████████████████████████████████| 536 kB 57.5 MB/s 
     |████████████████████████████████| 190 kB 48.1 MB/s 
  Created wheel for mmcv: filename=mmcv-1.5.2-py2.py3-none-any.whl size=817819 sha256=f3fe015779909e0ddd0c2499b1e0931cebb8513b925485b102a2b2373db6155e
  Stored in directory: /root/.cache/pip/wheels/f7/60/be/c3c92db989db1ca7055973f5913815c117b6a270e69e65b1d1
Successfully built mmcv


# Face tracking pipeline

The following example illustrates how to use the `facenet_pytorch` python package to perform face detection and tracking on an image dataset using MTCNN.

In [ ]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


#### Determine if an nvidia GPU is available

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

Note that, since MTCNN is a collection of neural nets and other code, the device must be passed in the following way to enable copying of objects when needed internally.

See `help(MTCNN)` for more details.

In [ ]:
mtcnn = MTCNN(keep_all=True, device=device)

#### Get a sample video

We begin by loading a video with some faces in it. The `mmcv` PyPI package by mmlabs is used to read the video frames (it can be installed with `pip install mmcv`). Frames are then converted to PIL images.

In [ ]:
! wget -nc https://github.com/timesler/facenet-pytorch/raw/master/examples/video.mp4

--2022-05-31 15:47:55--  https://github.com/timesler/facenet-pytorch/raw/master/examples/video.mp4
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/timesler/facenet-pytorch/master/examples/video.mp4 [following]
--2022-05-31 15:47:55--  https://raw.githubusercontent.com/timesler/facenet-pytorch/master/examples/video.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2411585 (2.3M) [application/octet-stream]
Saving to: ‘video.mp4’

video.mp4           100%[===================>]   2.30M  --.-KB/s    in 0.02s   

2022-05-31 15:47:55 (94.4 MB/s) - ‘video.mp4’ saved [2411585/2411585]



In [ ]:
from base64 import b64encode
from IPython.display import HTML

def displayVideo(file_path):
  mp4 = open(file_path, 'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
      <video width=800 controls>
            <source src="%s" type="video/mp4">
      </video>
    """ % data_url)

In [ ]:
video = mmcv.VideoReader('video.mp4')
frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

In [ ]:
displayVideo("video.mp4")

#### Run video through MTCNN

We iterate through each frame, detect faces, and draw their bounding boxes on the video frames.

In [ ]:
frames_tracked = []
for i, frame in enumerate(frames):
    print('\rTracking frame: {}'.format(i + 1), end='')
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame)
    
    # Draw faces
    frame_draw = frame.copy()
    draw = ImageDraw.Draw(frame_draw)
    for box in boxes:
        draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    
    # Add to frame list
    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
print('\nDone')

Tracking frame: 1

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Tracking frame: 98

#### Save tracked video

In [ ]:
dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'mp4v')    
video_tracked = cv2.VideoWriter('video_tracked.avi', fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()

In [ ]:
! pip install ffmpeg-python

In [ ]:
import ffmpeg
(
    ffmpeg
    .input('video_tracked.avi')
    .output('video_tracked.mp4')
    .run()
)

#### Display detections

In [ ]:
displayVideo("video_tracked.mp4")